In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from matrix_factorization import matrix_factorization
from graph_init import *
from similarity import *
from create_R import *
from ALS import *

In [2]:
# R = [
#      [5,3,0,1],
#      [4,0,0,1],
#      [1,1,0,5],
#      [1,0,0,4],
#      [0,1,5,4],
#     ]

# R = np.array(R)

R,R_dict = create_R()

print(R_dict)

/home/marc/Documents/MVA/ProjetGraphes/src/create_R.py:21: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  R[ratingsnp[i,0]-1, ratingsnp[i,-1]] = ratingsnp[i,2]


{'Ratings': array([ 2.5,  3. ,  3. , ...,  4. ,  2.5,  3.5]), 'Movies': array([   30.,   833.,   859., ...,  4597.,  4610.,  4696.]), 'Users': array([   0.,    0.,    0., ...,  670.,  670.,  670.])}


In [ ]:
N = len(R)
M = len(R[0])
K = 100

# P = np.random.rand(N,K)
# Q = np.random.rand(M,K)

# nP, nQ = matrix_factorization(R, P, Q, K)

als = ALS(K,N,M,"Users","Movies","Ratings")
print("Als created")
ans = als.fitImplicit(R_dict)

# nR = np.dot(nP, nQ.T)

# print(nP, "\n\n", nQ)

Als created


In [ ]:
als.U

In [ ]:
als.V

In [ ]:
R_rec = np.dot(als.U,np.transpose(als.V))
print(2*(R_rec-np.min(R_rec))*5/np.max(R_rec-np.min(R_rec))/2)

In [ ]:
R

In [ ]:
np.max(R_rec)

In [ ]:
lp = LaplacianParams()

sim = similarity(als.U)

print(sim)

In [ ]:
L = build_laplacian(sim,lp)

print(L)